In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
from utils.writer import save_offset_to_minio, write_to_parquet, write_to_postgres
from utils.reader import load_offset_from_minio
from confluent_kafka import Consumer
from utils.fetch_data import fetch_kafka_data
from utils.transform_data import transform_data
from clients.spark_client import create_spark_session
from db_utils.check_gp import create_db_schema_table
from clients.postgres_client import get_postgres_properties
from config import MINIO_RAW_PATH, MINIO_PROCESSED_PATH, DB_TABLE

In [41]:
create_db_schema_table()

База данных weather уже существует.
Схема weather уже существует.
Схема и таблица успешно созданы.


In [42]:
offset = load_offset_from_minio()
print(offset)

7


In [10]:
spark = create_spark_session()


In [ ]:
last_offset = fetch_kafka_data(spark, MINIO_RAW_PATH, offset)
save_offset_to_minio(last_offset)
print(last_offset)

[{'country': 'RU', 'city': 'Moscow', 'timestamp': '2025-06-04T13:55:55', 'temperature': 296.29, 'feels_like': 295.7, 'pressure': 1012, 'humidity': 40, 'weather_main': 'Clouds', 'weather_description': 'overcast clouds', 'wind_speed': 4.94, 'wind_deg': 343, 'clouds': 100}, {'country': 'RU', 'city': 'Moscow', 'timestamp': '2025-06-04T14:07:44', 'temperature': 296.24, 'feels_like': 295.65, 'pressure': 1012, 'humidity': 40, 'weather_main': 'Clouds', 'weather_description': 'overcast clouds', 'wind_speed': 4.94, 'wind_deg': 343, 'clouds': 100}, {'country': 'RU', 'city': 'Moscow', 'timestamp': '2025-06-04T14:07:44', 'temperature': 296.24, 'feels_like': 295.65, 'pressure': 1012, 'humidity': 40, 'weather_main': 'Clouds', 'weather_description': 'overcast clouds', 'wind_speed': 4.94, 'wind_deg': 343, 'clouds': 100}, {'country': 'RU', 'city': 'Moscow', 'timestamp': '2025-06-04T14:28:27', 'temperature': 295.04, 'feels_like': 294.25, 'pressure': 1012, 'humidity': 37, 'weather_main': 'Clouds', 'weathe

In [45]:
df_transform = transform_data(spark, MINIO_RAW_PATH, MINIO_PROCESSED_PATH)
write_to_parquet(spark, df_transform, MINIO_PROCESSED_PATH)


In [46]:
properties = get_postgres_properties()
write_to_postgres(df_transform, DB_TABLE, properties)